In [1]:
import numpy as np
import pandas as pd
from returns.result import Result, Success, Failure
from collections import Counter

In [2]:
#read the data. use the smallest training set to start
casp11 = "~/Downloads/casp11/training_30"
#read each line
with open(casp11, 'r') as casp:
    l = casp.read().split('\n')

In [3]:
#based on the formnat of primary, secondary, and tertiary, read in the data
primary = []
evolutionary = []
terti = []
ids = []

In [4]:
def line_read(blob):
    try:
        return [[float(x) for x in line.split('\t') if x!=""] for line in blob]
    except ValueError as vle:
        return Failure(vle)

In [7]:
#now process the data
#experiment with the first 1000 proteins
for x, line in enumerate(l):
    if len(ids) == 1000:
        break
    if line == "[ID]":
        ids.append(l[x+1])
    elif line == "[PRIMARY]":
        primary.append(l[x+1]) 
    elif line == "[EVOLUTIONARY]": #indexes based on explorations of the data format
        evolutionary.append(line_read(l[x+1:x+22]))
    elif line == "[TERTIARY]":
        terti.append(line_read(l[x+1:x+4]))
    else:
        pass

In [8]:
[len(x) for x in [ids, primary, evolutionary, terti]]

[1000, 999, 999, 999]

In [9]:
#check data attributes
#comprehensive check for length of each list in all read-in primary, evolutionary, and tertiary data
#clearly there are some 3D arrays there in our data

print(Counter([len(x) for x in [primary, evolutionary, terti]]))
print(Counter([len(y) for x in [primary, evolutionary, terti] for y in x]))
print(Counter([len(y) for x in [primary, evolutionary, terti] for y in x[998]]))
print(Counter([len(y) for x in terti for y in x]))
print(Counter([len(y) for x in [primary, evolutionary, terti] for y in x]))

Counter({1: 257, 257: 21, 771: 3})

In [26]:
#with the data, build the coordinates
#seqs - primary
#pssms - evolutionary (TODO: i know we are getting pssm matrices out of the 2ndary data, but not completely certin how. So i will do the tertiary data first.)
#coords - tertiary - we basically have a list of coordinates in this dataset, and we need to find some kind of angles

def build_crdnt(crdnt_array, remainder): 
#remainder can be: 0,1,2
    crdnt_list = []
    array_len = len(crdnt_array[1])
    for i in range(array_len):
        if i%3 == remainder:
            crdnt_list.append([crdnt_array[j][i] for j in range(3)])
    return np.array(crdnt_list)

In [27]:
#get all the coordinates
crdnt1 = [build_crdnt(crdnt,0) for crdnt in terti] 
crdnt2 = [build_crdnt(crdnt,1) for crdnt in terti]
crdnt3 = [build_crdnt(crdnt,2) for crdnt in terti]

In [28]:
#common sense check to make sure things work
[len(crdnt3 ), len(crdnt3 [1]), len(crdnt3[1][1])]

[999, 372, 3]

In [33]:
#form the coordinates, construct angles, the usual way
#c for coordinate. a for angle

def build_angle(c1,c2,c3,c4):
    a1=c2-c1
    a2=c3-c2
    a3=c4-c3
    
    v1=np.cross(a1,a2)
    v2=np.cross(a2, a3)
    
    v1= v1 / (v1 ** 2).sum(-1)**(0.5)
    v2= v2 / (v2 ** 2).sum(-1)**(0.5)
    
    pm=np.sign((a3*v1).sum(-1))
    denom= ((v1**2).sum(-1) * (v2**2).sum(-1))**0.5
    numerator=(v1*v2).sum(-1)
    angle=np.arccos(numerator/ denom)
    
    if pm!=0:
        angle=angle*pm
    return angle

In [62]:
#get the phis and the psis
#phi starts and psi ends with 0

thephis = []
thepsis=[]

clen = len(terti)

for i in range(clen):
    phi=[0.0]
    psi=[]
    len2=len(crdnt2[i])
    
    #TODO: for the 1st and last coordinates, there seem to be problems computing phi and psi, respectively. 
    for j in range(1,len2):
        phi_angle = build_angle(crdnt3[i][j-1], crdnt1[i][j], crdnt2[i][j], crdnt3[i][j]) 
        phi.append(phi_angle)
        if j < len2 - 1:
            psi_angle = build_angle(crdnt1[i][j], crdnt2[i][j], crdnt3[i][j], crdnt2[i][j+1]) 
            psi.append(psi_angle)
        else:
            pass
    
    psi.append(0.0)
    thephis.append(phi)
    thepsis.append(psi)

/home/hj/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':
/home/hj/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


In [68]:
#check
import gc
gc.collect()

513

In [70]:
print([len(thepsis), len(thephis)])

(999, 999)

In [11]:
# references: https://en.wikipedia.org/wiki/Amino_acid, section "Table of standard amino acid abbreviations and properties"
# references: https://en.wikipedia.org/wiki/Proteinogenic_amino_acid, section "General chemical properties“
#[chr(x) for x in range(ord('A'), ord('Z') + 1)]
all_aa = ['A','C','D','E','F','G','H','I','K','L','M','N','O','P','Q','R','S','T','U','V','W','Y']
#TODO: my hunch is that we need certain proproties - physical and chemical - of the amino acids, but I'm not sure what those properties are
#aa = amino acids
#aa_props1 = {}
#aa_props2 = {}

In [1]:
#TODO: I'm not totally certain. It seems like there are position and padding parameters involved. this is my current effort
#references: https://github.com/aqlaboratory/proteinnet/blob/master/docs/proteinnet_records.md
#https://en.wikipedia.org/wiki/Position_weight_matrix

def process_pssm(pssm, position, padding):
    result=[]
    for x, line in enumerate(pssm):
        start_index = position - padding
        end_index = position + padding
        result.append(line[start_index:end_index])
    return result

In [ ]:
#TODO： I think this is right based on what I researched. Need to further make sure

clen=len(primary)
pssm_in=[]
#TODO: padding length undecided
padding_len = 20

pssm_input = []
pssm_output = []

mycount = 0 # Counter to ensure everythings fine

for i in range(clen): 
    if len(primary[i])>padding_len*2:
        mycount += len(seqs[i])-padding_len*2
        for j in range(padding_len,len(seqs[i])-padding_len):
            pssm_input.append(process_pssm(pssms[i], j))
            pssm_output.append([thephis[i][j], thepsis[i][j]])

In [ ]:
#TODO: 1. finish PSSM. 
#2. further check on if "secondary structure" spoken in the documentation is present in this dataset (I checked multiple times, and want to make sure)
#3. finish data processing and onto MLing! 